In [2]:
import pandas as pd
import numpy as np
import sklearn as sk
import math

In [3]:
dist = pd.read_csv('dist_data.csv', sep=';')

In [4]:
dist_clean = dist[dist['dist'] != "[[], []]"]

In [5]:
x = dist_clean.iloc[:, :-1].values
y = dist_clean.iloc[:, 1].values

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [7]:
def clean_sample(sample):
    sample_list = sample[0].replace(', [', '; [').split(';')
    sample_x = sample_list[0]
    sample_Px = sample_list[1]

    sample_x_list = []
    sample_Px_list = []
    
    for x in sample_x.split(','):
        sample_x_list.append(x.replace(' ', '').replace('[', '').replace(']', ''))
    for Px in sample_Px.split(','):
        sample_Px_list.append(Px.replace(' ', '').replace('[', '').replace(']', ''))

    dict_sample = {}
    for i in range(len(sample_x_list)):
        dict_sample[sample_x_list[i]] = sample_Px_list[i]
    return dict_sample

In [8]:
def all_keys_dataset1_exist_in_dataset2(dataset1, dataset2, threshold):
    count = 0
    total = len(dataset1.keys())
    for key in dataset1.keys():
        if dataset2.get(key) is None:
            pass
        else:
            count += 1

    if (count/total) * 100 >= float(threshold):
        #print(count, (count/total) * 100, float(threshold))
        return True
    else:
        return False

In [9]:
def calculate_distance(distance_func, min_precent_similarity):
    #count = 0
    result = []
    for i, sample_x_test in enumerate(x_test): 
        result_zikma = []
    
        for j, sample_x_train in enumerate(x_train):
            if all_keys_dataset1_exist_in_dataset2(clean_sample(sample_x_test), clean_sample(sample_x_train), min_precent_similarity):
                Px_list = []
                for xd1, Pxd1 in clean_sample(sample_x_test).items():
                    if xd1 == "" or Pxd1 == "":
                        continue
                    Pxd2 = clean_sample(sample_x_train).get(xd1)
                    if Pxd2 is None:
                        Pxd2 = 0.0
                    Px_list.append([Pxd1, Pxd2])
                    
                for xd2, Pxd2 in clean_sample(sample_x_train).items():
                    if xd2 == "" or Pxd2 == "":
                        continue
                    Pxd1 = clean_sample(sample_x_test).get(xd2)
                    if Pxd1 is None:
                        Px_list.append([0.0, Pxd2])

                
                sum = distance_func(Px_list)
                result_zikma.append([y_train[j], sum])
                #print(sum)
                
            else:
                continue
        
        result.append([sample_x_test ,result_zikma])
        
        # count += 1
        # if count > 1:
        #     break
    
    return result

In [10]:
def js(Px_list):
    for i in Px_list:
        Mx = (float(i[0]) + float(i[1]))/2
        i.append(Mx)

    PM_list = []
    QM_list = []
    for i in Px_list:
        PM_list.append([i[0], i[2]])
        QM_list.append([i[1], i[2]])

    sum1 = kl(PM_list)
    sum2 = kl(QM_list)

    return (sum1 + sum2)/2


In [11]:
def kl(Px_list):
    sum = 0 
    for i in Px_list:
        Pxd1 = i[0]
        Pxd2 = i[1]
        if float(Pxd1) == 0.0:
            sum += 0.0
            continue
        sum += float(Pxd1) * math.log10(float(Pxd1) / float(Pxd2))
        
    return sum

In [12]:
def Predict(all_distance_label, threshold):
    y_pred = []
    for i in range(len(x_test)):
        t = pd.DataFrame(all_distance_label[i][1], columns=['Label', 'Kl Result'])
        tmp = t[t['Kl Result'] < threshold]
        grouped_df = tmp.groupby(['Label']).size().reset_index(name='Count')
        sorted_df = grouped_df.sort_values(by=['Count'], ascending=False)
        
        if tmp.values.size == 0:
            y_pred.append(0)
        else:
            y_pred.append(sorted_df.head(1)['Label'].values[0])
    return y_pred

In [13]:
def diffrence_pred_test(y_pred, y_test):
    test = []
    count = 0
    for i, value1 in enumerate(y_pred):
        if value1 != y_test[i]:
            test.append([i, value1, y_test[i]])
            count+=1
    
    return pd.DataFrame(test, columns=['index', 'y_pred', 'y_test'])

In [14]:
def other_percent_similarity():
    final_result = []
    for i in range(30, 101): # one or zero
        final_result.append(calculate_distance(js, i))
    return final_result

In [ ]:
final = other_percent_similarity()

In [ ]:
#y_pred = Predict(final_result, 0.096)

In [ ]:
#diffrence_pred_test(y_pred, y_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
res = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(res)
res1 = classification_report(y_test, y_pred)
print("Classification Report:",)
print (res1)
res2 = accuracy_score(y_test,y_pred)
print("Accuracy:",res2)